In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from tqdm import tqdm

# sources
# https://gist.github.com/MichelleDalalJian/453c68e7fde2b2996c8b598c988c09d3
# https://stackoverflow.com/questions/16597265/appending-to-an-empty-dataframe-in-pandas
# https://stackoverflow.com/questions/12850345/how-do-i-combine-two-dataframes
# https://www.geeksforgeeks.org/create-a-dictionary-with-list-comprehension-in-python/
# https://stackoverflow.com/questions/38233838/selecting-second-child-using-beautifulsoup
# https://stackoverflow.com/questions/24108507/beautiful-soup-resultset-object-has-no-attribute-find-all
# https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class
# https://stackoverflow.com/questions/63177365/why-is-concat-adding-time-to-the-date-when-concat-two-dataframes
# https://stackoverflow.com/questions/12451997/beautifulsoup-gettext-from-between-p-not-picking-up-subsequent-paragraphs

In [43]:
MULT_COUNTY_JAIL_BOOKINGS_URL = "http://www.mcso.us/PAID/Home/SearchResults"
CSV_LOCATION = 'data/data.csv'

prev_data = pd.read_csv(CSV_LOCATION, index_col=False)

response = requests.post(MULT_COUNTY_JAIL_BOOKINGS_URL)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')
table_input = table.prettify()

table_to_df = pd.read_html(table_input, parse_dates=[1])[0]

table_to_df = table_to_df.sort_values(by=["Booking Date", "Name"], ascending=[False, True])
# newdf = pd.concat([prev_data.astype({'Booking Date': 'str'}), table_to_df.astype({'Booking Date': 'str'})])
# newdf = newdf.drop_duplicates().reset_index(drop=True).sort_values(by="Booking Date", ascending=False)
# newdf = newdf.drop_duplicates(keep=False).sort_values(by=["Booking Date", "Name"])

table_to_df.to_csv('data/updated.csv', index=False)

In [21]:
links = table.find_all('a')
INMATE_COLUMNS = ['SWIS ID', 'Name', 'Age', 'Gender', 'Race', 'Height', 'Weight', 'Hair', 'Eyes', 'Arresting Agency', 'Booking Date', 'Assigned Facility', 'Projected Release Date']
inmate_datas = []
jailed_data = pd.DataFrame(columns=INMATE_COLUMNS)
for link in tqdm(links[:3]):
    booking_url = link.get('href')
    inmate_link = "http://www.mcso.us" + booking_url
    inmate_response = requests.get(inmate_link)
    soup = BeautifulSoup(inmate_response.text, 'html.parser')
    table = soup.find('table')
    table_input = table.prettify()
    booking_table = pd.read_html(table_input)[0][1]
    
    charge_infos = soup.find(id='charge-info')
    
#     inmate_data = dict(zip(INMATE_COLUMNS, booking_table))
#     inmate_datas.append(inmate_data)
    
    
#     followed_link = urlopen(inmate_link).read()
#     booking_table = pd.read_html(followed_link)[0][1]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.22it/s]


In [55]:
inmate_datas

[{'SWIS ID': '828509',
  'Name': 'Acevedohernandez, Jairo',
  'Age': '20',
  'Gender': 'Male',
  'Race': 'White',
  'Height': '5 ft 7 in',
  'Weight': '157 lbs',
  'Hair': 'Black',
  'Eyes': 'Brown',
  'Arresting Agency': 'Portland Police, Other',
  'Booking Date': '07/06/2021 11:40 PM',
  'Assigned Facility': 'MCDC',
  'Projected Release Date': 'Unknown'},
 {'SWIS ID': '782353',
  'Name': 'Adams, Tawnisha Nicole',
  'Age': '26',
  'Gender': 'Female',
  'Race': 'White',
  'Height': '5 ft 7 in',
  'Weight': '180 lbs',
  'Hair': 'Blonde',
  'Eyes': 'Blue',
  'Arresting Agency': 'Portland Police, Central Precinct',
  'Booking Date': '06/09/2022 08:01 AM',
  'Assigned Facility': 'MCDC',
  'Projected Release Date': 'Unknown'},
 {'SWIS ID': '830510',
  'Name': 'Adjaj, Salwan Wesam',
  'Age': '43',
  'Gender': 'Male',
  'Race': 'White',
  'Height': '6 ft 1 in',
  'Weight': '205 lbs',
  'Hair': 'Brown',
  'Eyes': 'Green',
  'Arresting Agency': 'US Marshals Service',
  'Booking Date': '06/17/20

In [83]:
inmate_link = "http://www.mcso.us/PAID/Home/Booking/1519001"
inmate_response = requests.get(inmate_link)
soup = BeautifulSoup(inmate_response.text, 'html.parser')

charge_infos = soup.find(id='charge-info')
cases = charge_infos.select('div > h3')

charge_items = []
charges = charge_infos.select('div > div > ol')
for charge in charges:
    for el in charge.find_all('li'):
        charge_type = el.find('span').getText()
        charge_bail = el.find(class_='charge-bail-display').text
        charge_status = el.find(class_='charge-status-value').text
        charge_item = {
            'Type': charge_type,
            'Bail': charge_bail,
            'Status': charge_status
        }
        charge_items.append(charge_item)

charge_items
        




[{'Type': 'MURDER II (A Felony)', 'Bail': 'Bail: $0', 'Status': 'Released'},
 {'Type': 'RECKLESS DRIVING (A Misdemeanor)',
  'Bail': 'Bail: $0',
  'Status': 'Released'},
 {'Type': 'RECK ENDANGER (A Misdemeanor)',
  'Bail': 'Bail: $0',
  'Status': 'Released'},
 {'Type': 'RECK ENDANGER (A Misdemeanor)',
  'Bail': 'Bail: $0',
  'Status': 'Released'},
 {'Type': 'TAMP PHY EVIDENCE (A Misdemeanor)',
  'Bail': 'Bail: $0',
  'Status': 'Released'},
 {'Type': 'MURDER II (A Felony)', 'Bail': 'Bail: $0', 'Status': 'Unsentenced'},
 {'Type': 'ATT COERCION (A Misdemeanor)',
  'Bail': 'Bail: $2,500',
  'Status': 'Unsentenced'},
 {'Type': 'RECKLESS DRIVING (A Misdemeanor)',
  'Bail': 'Bail: $2,500',
  'Status': 'Unsentenced'},
 {'Type': 'RECK ENDANGER (A Misdemeanor)',
  'Bail': 'Bail: $2,500',
  'Status': 'Unsentenced'},
 {'Type': 'RECK ENDANGER (A Misdemeanor)',
  'Bail': 'Bail: $2,500',
  'Status': 'Unsentenced'},
 {'Type': 'RECK ENDANGER (A Misdemeanor)',
  'Bail': 'Bail: $2,500',
  'Status': 'Unse

In [38]:
df1=pd.DataFrame({'A':[1,2,3,3],'B':[2,3,4,4]})
df2=pd.DataFrame({'A':[1],'B':[2]})
# res = pd.concat([df1, df2]).drop_duplicates(keep=False)
res = df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]

print(df1)
print(df2)

res

   A  B
0  1  2
1  2  3
2  3  4
3  3  4
   A  B
0  1  2


,A,B
1,2,3
2,3,4
3,3,4
